# Tensorflow Tutorial

## What is a Tensor?

Conceptually, a Tensor is a multi-dimensional data container for modern machine learning. Similar to NumPy ndarray objects, tf.Tensor objects have a data type and a shape. The main difference is that tf.Tensors, additionally, can reside in accelerator memory (like a GPU). TensorFlow offers a rich library of operations (tf.add, tf.matmul, tf.linalg.inv etc.) that consume and produce tf.Tensors.

Reference: https://hackernoon.com/learning-ai-if-you-suck-at-math-p4-tensors-illustrated-with-cats-27f0002c9b32

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# a rank 0 tensor; a scalar with shape []
print(3)

# a rank 1 tensor; a vector with shape [3]
print([1., 2., 3.])

# a rank 2 tensor; a matrix with shape [2, 3]
print([[1., 2., 3.], [4., 5., 6.]])

# a rank 3 tensor with shape [2, 1, 3]
print([[[1., 2., 3.]], [[7., 8., 9.]]])

3
[1.0, 2.0, 3.0]
[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]
[[[1.0, 2.0, 3.0]], [[7.0, 8.0, 9.0]]]


## Graph

A **computational graph** is a series of TensorFlow operations arranged into a graph. The graph is composed of two types of objects.

- Operations: The nodes of the graph. Operations describe calculations that consume and produce tensors.
- Tensors: The edges in the graph. These represent the values that will flow through the graph. Most TensorFlow functions return `tf.Tensors`.

In [3]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant([4,5]) # also tf.float32 implicitly
total = tf.add(b, 1)
print(a)
print(b)
print(total)

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor([4 5], shape=(2,), dtype=int32)
tf.Tensor([5 6], shape=(2,), dtype=int32)


**tf.Tensors work congruently with Numpy arrays (in eager execution)**. NumPy operations accept tf.Tensor arguments. TensorFlow math operations convert Python objects and NumPy arrays to tf.Tensor objects. The tf.Tensor.numpy method returns the object's value as a NumPy ndarray.

In [23]:
a = tf.constant([[1, 2], [3, 4]])
print('Tensor:', a)
print('Tensor shape:', a.shape)
print('Tensor type:', a.dtype)

Tensor: tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
Tensor shape: (2, 2)
Tensor type: <dtype: 'int32'>


In [25]:
# Broadcasting support
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

# Operator overloading is also supported
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


As seen from previous examples, tensors and Numpy arrays work congruently with each other. **We will show that we can also initialize a tensor using Numpy.** The function to be used is `tf.convert_to_tensor`

In [10]:
# create a Python array:
array_1d = np.array([1.3, 1, 4.0, 23.5])
tf_tensor = tf.convert_to_tensor(value=array_1d, dtype=tf.float64)

print(tf_tensor)
print('0:',tf_tensor[0])
print('2:',tf_tensor[2])

tf.Tensor([ 1.3  1.   4.  23.5], shape=(4,), dtype=float64)
0: tf.Tensor(1.3, shape=(), dtype=float64)
2: tf.Tensor(4.0, shape=(), dtype=float64)


## Tensorboard

TensorFlow provides a utility called TensorBoard. One of TensorBoard's many capabilities is visualizing a computation graph. You can easily do this with a few simple commands.

more detailed instructions and tutorial on: https://www.tensorflow.org/guide/summaries_and_tensorboard

## Constant/Variable

Variables is a primitive class in Tensorflow. It has the following properties:

* initial value is required
* updated during training
* in-memory buffer (saved/restored from disk)
* can be shared in a distributed environment
* they hold learned parameters of a model

In [28]:
# Let's try to compare with tf.Variable
a = tf.Variable([[1, 2], [3, 4]])
b = tf.Variable([[2, 2], [3, 3]])

print('Tensorflow variable:')
print(a)
print('Numpy variable:')
print(a.numpy())

# tf.multiply of two tf.Variable's 
print('Tensorflow multiplication:')
c_tf = tf.multiply(a, b)
print(c_tf)

print('Numpy muliplication:')
c_np = np.multiply(a, b)
print(c_np)

Tensorflow variable:
<tf.Variable 'Variable:0' shape=(2, 2) dtype=int32, numpy=
array([[1, 2],
       [3, 4]])>
Numpy variable:
[[1 2]
 [3 4]]
Tensorflow multiplication:
tf.Tensor(
[[ 2  4]
 [ 9 12]], shape=(2, 2), dtype=int32)
Numpy muliplication:
tf.Tensor(
[[ 2  4]
 [ 9 12]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[ 2  4]
 [ 9 12]], shape=(2, 2), dtype=int32)


In [31]:
# Don't do this! yields weird answers
d = tf.add(a,1)
d_np = np.multiply(a, d)
print(d_np)

[[<tf.Tensor: id=243, shape=(2, 2), dtype=int32, numpy=
array([[2, 4],
       [6, 8]])>
  <tf.Tensor: id=246, shape=(2, 2), dtype=int32, numpy=
array([[ 3,  6],
       [ 9, 12]])>]
 [<tf.Tensor: id=249, shape=(2, 2), dtype=int32, numpy=
array([[ 4,  8],
       [12, 16]])>
  <tf.Tensor: id=252, shape=(2, 2), dtype=int32, numpy=
array([[ 5, 10],
       [15, 20]])>]]
